In [3]:
%pip install dlt dlt[duckdb]

  Using cached pendulum-3.0.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
  Using cached pluggy-1.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
  Using cached time_machine-2.16.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.3/884.3 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 67.4 MB/s eta 0:00:00 MB/s eta 0:00:01
Using cached pendulum-3.0.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (384 kB)
Using cached pluggy-1.5.0-py3-none-any.whl (20 kB)
Using cached tomlkit-0.13.2-py3-none-any.whl (37 kB)
Using cached time_machine-2.16.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (32 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

In [6]:
@dlt.resource(name="rides")
def get_taxi_data():
    client = RESTClient(
        base_url="https://us-central1-dlthub-analytics.cloudfunctions.net/",
        paginator=PageNumberPaginator(
            base_page=1,
            total_path=None,
        ),
    )

    for page in client.paginate("data_engineering_zoomcamp_api"):
        yield page


pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_pipeline", destination="duckdb", dataset_name="ny_taxi_data"
)

load_info = pipeline.run(get_taxi_data)
print(load_info)

Pipeline ny_taxi_pipeline load step completed in 1.31 seconds
1 load package(s) were loaded to destination duckdb and into dataset ny_taxi_data
The duckdb destination used duckdb:////home/gcaru/code/data-engineering-zoomcamp/workshop-1/ny_taxi_pipeline.duckdb location to store data
Load package 1739390229.1090364 is LOADED and contains no failed jobs


In [8]:
import duckdb

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()

database        schema                 name  \
0  ny_taxi_pipeline  ny_taxi_data           _dlt_loads   
1  ny_taxi_pipeline  ny_taxi_data  _dlt_pipeline_state   
2  ny_taxi_pipeline  ny_taxi_data         _dlt_version   
3  ny_taxi_pipeline  ny_taxi_data                rides   

                                        column_names  \
0  [load_id, schema_name, status, inserted_at, sc...   
1  [version, engine_version, pipeline_name, state...   
2  [version, engine_version, inserted_at, schema_...   
3  [end_lat, end_lon, fare_amt, passenger_count, ...   

                                        column_types  temporary  
0  [VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...      False  
1  [BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...      False  
2  [BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...      False  
3  [DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...      False

In [9]:
df = pipeline.dataset(dataset_type="default").rides.df()
df

end_lat    end_lon  fare_amt  passenger_count payment_type  start_lat  \
0     40.742963 -73.980072      45.0                1       Credit  40.641525   
1     40.740187 -74.005698       6.5                1       Credit  40.722065   
2     40.718043 -74.004745      12.5                5       Credit  40.761945   
3     40.739637 -73.985233       4.9                1         CASH  40.749802   
4     40.730032 -73.852693      25.7                1         CASH  40.776825   
...         ...        ...       ...              ...          ...        ...   
9995  40.783522 -73.970690       5.7                1         CASH  40.778560   
9996  40.777200 -73.964197       4.1                1         CASH  40.779800   
9997  40.780172 -73.957617       6.1                1         CASH  40.788388   
9998  40.777342 -73.957242       5.7                1         CASH  40.773828   
9999  40.757122 -73.986293       6.5                1       Credit  40.756313   

      start_lon  tip_amt  tolls_amt  total_amt  trip_distance  \
0    -73.787442      9.0       4.15      58.15          17.52   
1    -74.009767      1.0       0.00       8.50           1.56   
2    -73.983038      2.0       0.00      15.50           3.37   
3    -73.992247      0.0       0.00       5.40           1.11   
4    -73.949233      0.0       4.15      29.85          11.09   
...         ...      ...        ...        ...            ...   
9995 -73.953660      0.0       0.00       5.70           1.16   
9996 -73.974297      0.0       0.00       4.10           0.89   
9997 -73.976758      0.0       0.00       6.10           1.30   
9998 -73.956690      0.0       0.00       6.20           0.97   
9999 -73.972948      2.0       0.00       8.50           0.92   

        trip_dropoff_date_time     trip_pickup_date_time  surcharge  \
0    2009-06-14 23:48:00+00:00 2009-06-14 23:23:00+00:00        0.0   
1    2009-06-18 17:43:00+00:00 2009-06-18 17:35:00+00:00        1.0   
2    2009-06-10 18:27:00+00:00 2009-06-10 18:08:00+00:00        1.0   
3    2009-06-14 23:58:00+00:00 2009-06-14 23:54:00+00:00        0.5   
4    2009-06-13 13:23:00+00:00 2009-06-13 13:01:00+00:00        0.0   
...                        ...                       ...        ...   
9995 2009-06-19 11:28:00+00:00 2009-06-19 11:22:00+00:00        0.0   
9996 2009-06-17 07:43:00+00:00 2009-06-17 07:41:00+00:00        0.0   
9997 2009-06-19 11:46:00+00:00 2009-06-19 11:39:00+00:00        0.0   
9998 2009-06-17 04:19:00+00:00 2009-06-17 04:13:00+00:00        0.5   
9999 2009-06-17 08:34:00+00:00 2009-06-17 08:24:00+00:00        0.0   

     vendor_name        _dlt_load_id         _dlt_id  store_and_forward  
0            VTS  1739390229.1090364  1/ZSM1liNtd7hg                NaN  
1            VTS  1739390229.1090364  gKbCn/BZ7P+j5A                NaN  
2            VTS  1739390229.1090364  6bD4CCA+Cf8dCg                NaN  
3            VTS  1739390229.1090364  molDAeYankicPw                NaN  
4            VTS  1739390229.1090364  Zi8LzaESPthMjQ                NaN  
...          ...                 ...             ...                ...  
9995         VTS  1739390229.1090364  gDSj57zWaQeawA                NaN  
9996         VTS  1739390229.1090364  qAwRt3XNpQrmvw                NaN  
9997         VTS  1739390229.1090364  yKx9oAU+Lh09jw                NaN  
9998         VTS  1739390229.1090364  Ok4DUk5pWl7Lkg                NaN  
9999         VTS  1739390229.1090364  Hphf6yUWwYA39w                NaN  

[10000 rows x 18 columns]

In [10]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM rides;
            """
        )
    print(res)

[(12.3049,)]
